In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
data = pd.read_excel('Data.xlsx')

In [3]:
data.head(5)

,Unnamed: 0,Agent_id,status,lost_reason,budget,lease,movein,source,source_city,source_country,utm_source,utm_medium,des_city,des_country,room_type,lead_id
0,0,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Not responding,NaN,NaN,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,cd5dc0d9393f3980d11d4ba6f88f8110c2b7a7f7796307...
1,1,299ae77a4ef350ae0dd37d6bba1c002d03444fb1edb236...,LOST,Low budget,NaN,NaN,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,5372372f3bf5896820cb2819300c3e681820d82c6efc54...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,b94693673a5f7178d1b114e4004ad52377d3244dd24a3d...
2,2,c213697430c006013012dd2aca82dd9732aa0a1a6bca13...,LOST,Not responding,£121 - £180 Per Week,Full Year Course Stay 40 - 44 weeks,2022-08-31,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,9b8cc3c63cdf447e463c11544924bf027945cbd29675f7...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,11ab03a1a8c367191355c152f39fe28cae5e426fce49ef...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Ensuite,96ea4e2bf04496c044745938c0299c264c3f4ba079e572...
3,3,eac9815a500f908736d303e23aa227f0957177b0e6756b...,LOST,Low budget,0-0,0,NaT,ba2d0a29556ac20f86f45e4543c0825428cba33fd7a9ea...,a5f0d2d08eb0592087e3a3a2f9c1ba2c67cc30f2efd2bd...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,19372fa44c57a01c37a5a8418779ca3d99b0b59731fb35...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,1d2b34d8add02a182a4129023766ca4585a8ddced0e5b3...
4,4,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Junk lead,NaN,NaN,NaT,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,NaN,fc10fffd29cfbe93c55158fb47752a7501c211d253468c...


In [4]:
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

In [5]:
data.drop_duplicates(subset=['lead_id'], keep='first', inplace=True)
len(data['lead_id'].unique())

30574

In [6]:
#extracted year from movein
data['movein_year'] = data['movein'].dt.year
data.drop('movein', axis=1, inplace=True)

In [7]:
data.head()

,Agent_id,status,lost_reason,budget,lease,source,source_city,source_country,utm_source,utm_medium,des_city,des_country,room_type,lead_id,movein_year
0,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Not responding,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,cd5dc0d9393f3980d11d4ba6f88f8110c2b7a7f7796307...,NaN
1,299ae77a4ef350ae0dd37d6bba1c002d03444fb1edb236...,LOST,Low budget,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,5372372f3bf5896820cb2819300c3e681820d82c6efc54...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,b94693673a5f7178d1b114e4004ad52377d3244dd24a3d...,NaN
2,c213697430c006013012dd2aca82dd9732aa0a1a6bca13...,LOST,Not responding,£121 - £180 Per Week,Full Year Course Stay 40 - 44 weeks,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,9b8cc3c63cdf447e463c11544924bf027945cbd29675f7...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,11ab03a1a8c367191355c152f39fe28cae5e426fce49ef...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Ensuite,96ea4e2bf04496c044745938c0299c264c3f4ba079e572...,2022.0
3,eac9815a500f908736d303e23aa227f0957177b0e6756b...,LOST,Low budget,0-0,0,ba2d0a29556ac20f86f45e4543c0825428cba33fd7a9ea...,a5f0d2d08eb0592087e3a3a2f9c1ba2c67cc30f2efd2bd...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,19372fa44c57a01c37a5a8418779ca3d99b0b59731fb35...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,1d2b34d8add02a182a4129023766ca4585a8ddced0e5b3...,NaN
4,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Junk lead,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,NaN,fc10fffd29cfbe93c55158fb47752a7501c211d253468c...,NaN


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30574 entries, 0 to 46607
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Agent_id        30574 non-null  object 
 1   status          30574 non-null  object 
 2   lost_reason     28348 non-null  object 
 3   budget          28212 non-null  object 
 4   lease           29079 non-null  object 
 5   source          30574 non-null  object 
 6   source_city     30574 non-null  object 
 7   source_country  30574 non-null  object 
 8   utm_source      30574 non-null  object 
 9   utm_medium      30574 non-null  object 
 10  des_city        30574 non-null  object 
 11  des_country     30574 non-null  object 
 12  room_type       15248 non-null  object 
 13  lead_id         30574 non-null  object 
 14  movein_year     21552 non-null  float64
dtypes: float64(1), object(14)
memory usage: 3.7+ MB


In [9]:
data.shape

(30574, 15)

In [10]:
data.isnull().sum()

Agent_id              0
status                0
lost_reason        2226
budget             2362
lease              1495
source                0
source_city           0
source_country        0
utm_source            0
utm_medium            0
des_city              0
des_country           0
room_type         15326
lead_id               0
movein_year        9022
dtype: int64

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30574 entries, 0 to 46607
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Agent_id        30574 non-null  object 
 1   status          30574 non-null  object 
 2   lost_reason     28348 non-null  object 
 3   budget          28212 non-null  object 
 4   lease           29079 non-null  object 
 5   source          30574 non-null  object 
 6   source_city     30574 non-null  object 
 7   source_country  30574 non-null  object 
 8   utm_source      30574 non-null  object 
 9   utm_medium      30574 non-null  object 
 10  des_city        30574 non-null  object 
 11  des_country     30574 non-null  object 
 12  room_type       15248 non-null  object 
 13  lead_id         30574 non-null  object 
 14  movein_year     21552 non-null  float64
dtypes: float64(1), object(14)
memory usage: 3.7+ MB


In [12]:
data = data.drop(['lead_id'], axis=1)

In [13]:
#remove status other than ‘WON’ or ‘LOST’
data = data[data['status'].isin(['WON', 'LOST'])]

In [14]:
data['status'].unique()

array(['LOST', 'WON'], dtype=object)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30382 entries, 0 to 46607
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Agent_id        30382 non-null  object 
 1   status          30382 non-null  object 
 2   lost_reason     28348 non-null  object 
 3   budget          28024 non-null  object 
 4   lease           28890 non-null  object 
 5   source          30382 non-null  object 
 6   source_city     30382 non-null  object 
 7   source_country  30382 non-null  object 
 8   utm_source      30382 non-null  object 
 9   utm_medium      30382 non-null  object 
 10  des_city        30382 non-null  object 
 11  des_country     30382 non-null  object 
 12  room_type       15091 non-null  object 
 13  movein_year     21377 non-null  float64
dtypes: float64(1), object(13)
memory usage: 3.5+ MB


In [16]:
#replacing to NaN
data.replace('9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0', np.nan, inplace=True)

In [17]:
features_with_na=[features for features in data.columns if data[features].isnull().sum()>1]

for feature in features_with_na:
    print(feature, np.round(data[feature].isnull().mean(), 4),  ' % missing values')

lost_reason 0.0669  % missing values
budget 0.0776  % missing values
lease 0.0491  % missing values
source 0.1291  % missing values
source_city 0.1808  % missing values
source_country 0.1764  % missing values
utm_source 0.0016  % missing values
utm_medium 0.0691  % missing values
des_city 0.0555  % missing values
des_country 0.0555  % missing values
room_type 0.5033  % missing values
movein_year 0.2964  % missing values


In [18]:
data.fillna(data.mode().iloc[0], inplace=True)

In [19]:
data.isnull().sum()

Agent_id          0
status            0
lost_reason       0
budget            0
lease             0
source            0
source_city       0
source_country    0
utm_source        0
utm_medium        0
des_city          0
des_country       0
room_type         0
movein_year       0
dtype: int64

In [20]:
data.columns

Index(['Agent_id', 'status', 'lost_reason', 'budget', 'lease', 'source',
       'source_city', 'source_country', 'utm_source', 'utm_medium', 'des_city',
       'des_country', 'room_type', 'movein_year'],
      dtype='object')

In [21]:
# Convert categorical features to numerical using one-hot encoding
data = pd.get_dummies(data, columns=['Agent_id', 'lost_reason', 'budget', 'lease', 'source',
       'source_city', 'source_country', 'utm_source', 'utm_medium', 'des_city',
       'des_country', 'room_type'], drop_first=True)

In [22]:
#Target variable
import category_encoders as ce
mapping = [{'col': 'status', 'mapping': {"WON": 1,  "LOST": 0}}]
data = ce.OrdinalEncoder(cols=['status'], mapping=mapping).fit(data).transform(data)

In [23]:
data['status'].head(10)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
9     0
11    0
12    0
Name: status, dtype: int32

In [24]:
#Split the dataset into training and testing datasets
X = data.drop('status', axis=1)
y = data['status']

In [25]:
#Feature selection
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [26]:
#Apply Feature Selection
feature_sel_model = SelectFromModel(Lasso(alpha=0.005, random_state=0)) # remember to set the seed, the random state in this function
feature_sel_model.fit(X, y)

SelectFromModel(estimator=Lasso(alpha=0.005, random_state=0))

In [27]:
feature_sel_model.get_support()

array([False, False, False, ..., False, False, False])

In [28]:
#let's print the number of total and selected features

#this is how we can make a list of the selected features
selected_feat = X.columns[(feature_sel_model.get_support())]

print('total features: {}'.format((X.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(feature_sel_model.estimator_.coef_ == 0)))

total features: 6205
selected features: 4
features with coefficients shrank to zero: 6201


In [29]:
selected_feat

Index(['Agent_id_2fca346db656187102ce806ac732e06a62df0dbb2829e511a770556d398e1a6e',
       'lost_reason_Low availability',
       'utm_medium_09076eb7665d1fb9389c7c4517fee0b00e43092eb34821b09b5730c41ebcc50c',
       'des_city_ecc0e7dc084f141b29479058967d0bc07dee25d9690a98ee4e6fdad5168274d7'],
      dtype='object')

In [30]:
X = X[selected_feat]

In [31]:
X.head()

,Agent_id_2fca346db656187102ce806ac732e06a62df0dbb2829e511a770556d398e1a6e,lost_reason_Low availability,utm_medium_09076eb7665d1fb9389c7c4517fee0b00e43092eb34821b09b5730c41ebcc50c,des_city_ecc0e7dc084f141b29479058967d0bc07dee25d9690a98ee4e6fdad5168274d7
0,0,0,0,1
1,0,0,0,0
2,0,0,1,0
3,0,0,1,0
4,0,0,0,1


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
#Train a Random Forest Classifier model
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

In [34]:
y_pred=rf_pred
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.981898963304262
Precision: 0.9830710562202366
Recall: 0.981898963304262
F1-Score: 0.9823192923665367


In [35]:
#Train a Decision Tree Classifier model
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)

In [36]:
y_pred=dt_pred
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.981898963304262
Precision: 0.9830710562202366
Recall: 0.981898963304262
F1-Score: 0.9823192923665367


In [37]:
#Train a Logistic Regression model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

In [38]:
y_pred=lr_pred
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.981898963304262
Precision: 0.9830710562202366
Recall: 0.981898963304262
F1-Score: 0.9823192923665367
